In [ ]:
import os
import sys

# Obtenir le répertoire de travail courant
current_dir = os.getcwd()

# Obtenir le répertoire parent de `maps` (qui est `src`)
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

# Ajouter `src` au chemin de recherche des modules
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [ ]:
import pandas as pd
import numpy as np
import folium.features
from sklearn.cluster import DBSCAN
from sklearn.cluster import HDBSCAN
from sklearn.cluster import OPTICS
from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import PolynomialFeatures
from pyproj import Transformer
import seaborn as sns

from python_scripts.road_detection.road_utils import *
from python_scripts.miscellaneaous.data_processing import extract_data
from python_scripts.graphs.graphs_creation import delaunay_graph
from python_scripts.city.city_utils import plotMapWithColors, mean_distance_choice, mean_distance_to_NN
from python_scripts.ihm.maps.mapUtils import *

In [ ]:
pwdMap="../../out/maps/"

In [ ]:
df = pd.read_csv("../../database/data.csv", sep=";", decimal=',')

In [ ]:
df_extracted = extract_data(df, provider='Orange', techno='4g')#, region='Normandie') #Occitanie, Île-de-France
G, pos = delaunay_graph(df_extracted)

In [ ]:
mean_distances = mean_distance_to_NN(df_extracted[['x', 'y']], n_neighbours=3) # 3 to have more neighbours

In [ ]:
mean_distance_params = {
    ']0, 1] km': {'colour': '#030464', 'angle': 40, 'distance': 2},
    ']1, 2] km': {'colour': '#069AF3', 'angle': 30, 'distance': 5},
    ']2, 4] km': {'colour': '#02D4BB', 'angle': 25, 'distance': 10},
    ']4, inf] km': {'colour': '#0DBF75', 'angle': 15, 'distance': 15},
}

In [ ]:
countryside = []
for station in mean_distances.index:
    if(mean_distances[station]>2):
        countryside.append(station)

In [ ]:
clust_dbscan = road_get_clust_dbscan(df_extracted, countryside)
clust_hdbscan = road_get_clust_hdbscan(df_extracted, countryside)
clust_optics = road_get_clust_optics(df_extracted, countryside)


In [ ]:
def rgb_to_hex(rgb):
        return '#{:02x}{:02x}{:02x}'.format(int(rgb[0]*255), int(rgb[1]*255), int(rgb[2]*255))

def labelToColor(clustId, clusters, palette):   
    if (clustId == -1):
        return 'gray'
    else:
        return rgb_to_hex(palette[list(clusters).index(clustId)- (1 if -1 in clusters else 0)])

In [ ]:
def applyPolynomialRegressionAndPlotMap(clust, df_extracted, countryside):
    clusters = clust.unique()
    num_clusters = len(clusters) - (1 if -1 in clusters else 0)

    excluded_clusters, not_exluded_clusters, linearModels, xBounds = detect_roads_based_on_clusters(clust, df_extracted)

    palette = sns.color_palette("hsv", num_clusters)
    colors = clust.apply(lambda clustId : labelToColor(clustId, clusters, palette))
    map = plotMapWithColorsAndLayers(df_extracted, countryside, colors, linearModels, xBounds, clust, [not_exluded_clusters, excluded_clusters, [-1]], layersLabel=["roads", "non-roads", "noise"], mapName="OpenStreetMap")
    return (excluded_clusters, not_exluded_clusters, map)

In [ ]:
map1=applyPolynomialRegressionAndPlotMap(clust_dbscan, df_extracted, countryside)[2]
map2=applyPolynomialRegressionAndPlotMap(clust_hdbscan, df_extracted, countryside)[2]
map3=applyPolynomialRegressionAndPlotMap(clust_optics, df_extracted, countryside)[2]

In [ ]:
map1.save(pwdMap + "road_regression_dbscan")
map2.save(pwdMap + "road_regression_hdbscan")
map3.save(pwdMap + "road_regression_optics")

In [ ]:
for clustering in [clust_dbscan, clust_hdbscan, clust_optics]:
    for cluster in clustering.loc[clustering!=-1].unique():
        nb_elem = list(clustering).count(cluster)
        if(nb_elem <= 10):
            clustering.loc[clustering==cluster] = -1

In [ ]:
possible_points = []
for bsId in countryside:
    if((clust_dbscan[bsId]!=-1) or (clust_dbscan[bsId]!=-1) or (clust_dbscan[bsId]!=-1)):
        possible_points.append(bsId)

In [ ]:
clust_dbscan = road_get_clust_dbscan(df_extracted, possible_points)
clust_hdbscan = road_get_clust_hdbscan(df_extracted, possible_points)
clust_optics = road_get_clust_optics(df_extracted, possible_points)

In [ ]:
map1=applyPolynomialRegressionAndPlotMap(clust_dbscan, df_extracted, possible_points)[2]
map2=applyPolynomialRegressionAndPlotMap(clust_hdbscan, df_extracted, possible_points)[2]
map3=applyPolynomialRegressionAndPlotMap(clust_optics, df_extracted, possible_points)[2]

In [ ]:
map1.save(pwdMap + "2steps_road_regression_dbscan")
map2.save(pwdMap + "2steps_road_regression_hdbscan")
map3.save(pwdMap + "2steps_road_regression_optics")